# Purpose:
This notebook contains the code that is use to generate weights for the three "Score prediction" regression neural nets. 

Each net is trained on a different type of score. The final output of this notebook is the saved keras weights which are used in the final predictor. 



In [9]:
!nvidia-smi

Thu Sep  7 19:52:42 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   43C    P0    62W / 250W |     10MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [10]:
import numpy as np
from keras import backend as K
from keras.layers import Input, Embedding, merge
import keras.layers
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#import pydotplus as pydot 
#import graphviz
from keras.utils import plot_model
from sklearn.metrics import mean_absolute_error

In [11]:
import pandas as pd

In [12]:
ratings_score_train = pd.read_csv('mal_scores_train_nonzero_v2.csv')#  Time to load in all of the data. 
ratings_score_test = pd.read_csv('mal_scores_test_nonzero_v2.csv')# the v2 version of this data has no "test" users who aren't present in the training data. 
#ratings_no_score_train = pd.read_csv('mal_scores_train_zero.csv') # In prior iterations of this model, this was used to impute scores in a stacked neural net. This did not improve the final recommendations. 

At this iteration, we can load in a dictionary of user an anime that have been vectorized. Commented out is the code needed to regenerate that dictionary. 


In [13]:
userid2idx=np.load("user.npy").item()
animeid2idx=np.load("anime.npy").item()
#From prior parts of the model, the user and anime embedding id is stored in this dictionary, to make sure that anime have the same embedding when they are loaded into the recommendation engine.  
#userid2idx = {o:i for i,o in enumerate(users)} # is the code redo the dictionary
#animeid2idx = {o:i for i,o in enumerate(animes)} # remove missing anime numbers and re-order
#np.save("user.npy", userid2idx)
#np.save("anime.npy", animeid2idx)
#userid mapping
#users1 = ratings_score_train.userid.unique()
#users2 = ratings_score_test.userid.unique()
#users3 = ratings_no_score_train.userid.unique()
#animeid mapping
#anime1= ratings_score_train.animeid.unique()
#anime2=ratings_score_test.animeid.unique()
#anime3=ratings_no_score_train.animeid.unique()
#animes = set(anime1).union(set(anime2)).union(set(anime3))
#users = set(users1).union(set(users2)).union(set(users3))


In [14]:
n_users = len(userid2idx)
n_animes = len(animeid2idx)

In [16]:
ratings_score_test['anime_id_emb'] = ratings_score_test.animeid.apply(lambda x: animeid2idx[x])
ratings_score_train['anime_id_emb'] = ratings_score_train.animeid.apply(lambda x: animeid2idx[x])
#ratings_no_score_train['anime_id_emb'] = ratings_no_score_train.animeid.apply(lambda x: animeid2idx[x])

In [17]:
n_users

230962

In [18]:
n_animes

12873

In [19]:
ratings_score_test['user_id_emb'] = ratings_score_test.userid.apply(lambda x: userid2idx[x])
ratings_score_train['user_id_emb'] = ratings_score_train.userid.apply(lambda x: userid2idx[x])
#ratings_no_score_train['user_id_emb'] = ratings_no_score_train.userid.apply(lambda x: userid2idx[x])

In [20]:
n_factors = 36 #changing this number changes how many hidden factors each user and each anime is transformed into. 

## Data Cleaning:
After some iteration, the final recommendations are improved by removing certain outliers. 

In [21]:
ratings_score_train=ratings_score_train[(ratings_score_train['user_rev_count']<1500) & (ratings_score_train['user_rev_count']>25)]

In [22]:
ratings_score_train=ratings_score_train[ratings_score_train['score_usr_scaled']!=0]

In [23]:
ratings_score_train = ratings_score_train[ratings_score_train['anime_rev_count']<50000]# let's remove the absolutely most common.

In [24]:
def embedding_input_anime1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in1, u1 = embedding_input_user1('user_id_in', n_users+15, n_factors, 1e-5)
anime_in1, a1 = embedding_input_anime1('anime_id_in', n_animes, n_factors, 0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(230977, 36, embeddings_regularizer=<keras.reg..., name="Embed_User_Hidden_Factors", input_length=1)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12873, 36, embeddings_regularizer=<keras.reg..., name="Embed_Anime_Hidden_Factors", input_length=1)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
# nn1, It is given no modifications for status. It is predicting score
x = merge([u1, a1], mode='concat', name='All_Factors_on_one_layer')
x = Flatten()(x)
#x = Dropout(0.55, name='Prevent_overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.55,name='Prevent_overfit2')(x)
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x = Dropout(0.1, name='Prevent_overfit')(x)
x = Dense(1,name='Final_Interactions')(x)
nn1 = Model([user_in1, anime_in1], x)
nn1.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [26]:
nn1.fit([ratings_score_train.user_id_emb, ratings_score_train.anime_id_emb], ratings_score_train.score, batch_size=5120, epochs=8, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score))

Train on 9771714 samples, validate on 1210873 samples
Epoch 1/8
9771714/9771714 [==============================] - 27s - loss: 4.9472 - val_loss: 2.0314
Epoch 2/8
9771714/9771714 [==============================] - 26s - loss: 2.3761 - val_loss: 1.9279
Epoch 3/8
9771714/9771714 [==============================] - 26s - loss: 2.2107 - val_loss: 1.8766
Epoch 4/8
9771714/9771714 [==============================] - 26s - loss: 2.0277 - val_loss: 1.8376
Epoch 5/8
9771714/9771714 [==============================] - 26s - loss: 1.8506 - val_loss: 1.8168
Epoch 6/8
9771714/9771714 [==============================] - 26s - loss: 1.7181 - val_loss: 1.8047
Epoch 7/8
9771714/9771714 [==============================] - 26s - loss: 1.6383 - val_loss: 1.7912
Epoch 8/8
9771714/9771714 [==============================] - 26s - loss: 1.6026 - val_loss: 1.7964


In [27]:
nn1.lr=.0005

In [28]:
nn1.fit([ratings_score_train.user_id_emb, ratings_score_train.anime_id_emb], ratings_score_train.score, batch_size=10120, epochs=8, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score))

Train on 9771714 samples, validate on 1210873 samples
Epoch 1/8
9771714/9771714 [==============================] - 16s - loss: 1.5587 - val_loss: 1.7777
Epoch 2/8
9771714/9771714 [==============================] - 16s - loss: 1.5503 - val_loss: 1.7758
Epoch 3/8
9771714/9771714 [==============================] - 15s - loss: 1.5460 - val_loss: 1.7798
Epoch 4/8
9771714/9771714 [==============================] - 16s - loss: 1.5394 - val_loss: 1.7796
Epoch 5/8
9771714/9771714 [==============================] - 15s - loss: 1.5337 - val_loss: 1.7785
Epoch 6/8
9771714/9771714 [==============================] - 16s - loss: 1.5275 - val_loss: 1.7799
Epoch 7/8
9771714/9771714 [==============================] - 15s - loss: 1.5228 - val_loss: 1.7785
Epoch 8/8
9771714/9771714 [==============================] - 15s - loss: 1.5178 - val_loss: 1.7833


In [29]:
nn1pred= nn1.predict([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb])
nn1targ = ratings_score_test.score.values
mean_absolute_error(nn1targ, nn1pred) 

0.98166690871267037

In [30]:
nn1.save_weights('nn_score_weights.h5')

In [30]:
#plot_model(nn1, to_file='score.png')
#plot_model(nn12, to_file='user_score.png')
#plot_model(nn13, to_file='anime_score.png')

The User_Scaled_Score branch
data prep: drop all scores for shows that are not  listed as complete or dropped.


In [21]:
ratings_score_train_complete= ratings_score_train[(ratings_score_train['status']=='COMPLETED') | (ratings_score_train['status']=='DROPPED')]

In [22]:
#ratings_score_train_complete= ratings_score_train # The alternative to the above cell to train the model on shows of all status. 

In [33]:
def embedding_input_anime22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in22, u22 = embedding_input_user22('user_id_in', n_users+15, n_factors, 1e-5)
anime_in22, a22 = embedding_input_anime22('anime_id_in', n_animes, n_factors, 0)
# nn22, It is only given complete. Trying to predict usr scaled score. 
x = merge([u22, a22], mode='concat', name='All_Factors_on_one_layer')
x = Flatten()(x)
#x = Dropout(0.55, name='Prevent_overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.55,name='Prevent_overfit2')(x)
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x = Dropout(0.1, name='Prevent_overfit')(x)
x = Dense(1,name='Final_Interactions')(x)
nn22 = Model([user_in22, anime_in22], x)
nn22.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(230977, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_User_Hidden_Factors")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12873, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_Anime_Hidden_Factors")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use inst

In [34]:
nn22.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_usr_scaled, batch_size=5120, epochs=8, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_usr_scaled))

Train on 9240364 samples, validate on 1210873 samples
Epoch 1/8
9240364/9240364 [==============================] - 24s - loss: 1.6491 - val_loss: 1.6467
Epoch 2/8
9240364/9240364 [==============================] - 24s - loss: 1.5799 - val_loss: 1.6337
Epoch 3/8
9240364/9240364 [==============================] - 23s - loss: 1.5548 - val_loss: 1.6252
Epoch 4/8
9240364/9240364 [==============================] - 23s - loss: 1.5373 - val_loss: 1.6246
Epoch 5/8
9240364/9240364 [==============================] - 24s - loss: 1.5243 - val_loss: 1.6240
Epoch 6/8
9240364/9240364 [==============================] - 23s - loss: 1.5136 - val_loss: 1.6218
Epoch 7/8
9240364/9240364 [==============================] - 24s - loss: 1.5052 - val_loss: 1.6226
Epoch 8/8
9240364/9240364 [==============================] - 23s - loss: 1.4980 - val_loss: 1.6211


In [35]:
nn22.lr=.0005

In [36]:
nn22.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_usr_scaled, batch_size=20120, epochs=4, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_usr_scaled))

Train on 9240364 samples, validate on 1210873 samples
Epoch 1/4
9240364/9240364 [==============================] - 10s - loss: 1.4488 - val_loss: 1.6171
Epoch 2/4
9240364/9240364 [==============================] - 10s - loss: 1.4322 - val_loss: 1.6151
Epoch 3/4
9240364/9240364 [==============================] - 10s - loss: 1.4273 - val_loss: 1.6157
Epoch 4/4
9240364/9240364 [==============================] - 10s - loss: 1.4244 - val_loss: 1.6178


In [37]:
nn22.save_weights('nn_score_usr_weights.h5')

In [38]:
nn22pred= nn22.predict([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb])
nn22targ = ratings_score_test.score_usr_scaled.values
mean_absolute_error(nn22targ, nn22pred)

0.93386003456007916

In [39]:
def embedding_input_anime23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in23, u23 = embedding_input_user23('user_id_in', n_users+15, n_factors, 0)
anime_in23, a23 = embedding_input_anime23('anime_id_in', n_animes, n_factors, 0)
# nn23, It is only given complete. Trying to predict anime scaled score. 
x = merge([u23, a23], mode='concat', name='All_Factors_on_one_layer')
x = Flatten()(x)
#x = Dropout(0.55, name='Prevent_overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.55,name='Prevent_overfit2')(x)
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x = Dropout(0.1, name='Prevent_overfit')(x)
x = Dense(1,name='Final_Interactions')(x)
nn23 = Model([user_in23, anime_in23], x)
nn23.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(230977, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_User_Hidden_Factors")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12873, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_Anime_Hidden_Factors")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use inst

In [40]:
nn23.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_anime_scaled, batch_size=5120, epochs=8, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_anime_scaled))

Train on 9240364 samples, validate on 1210873 samples
Epoch 1/8
9240364/9240364 [==============================] - 21s - loss: 1.7164 - val_loss: 1.7234
Epoch 2/8
9240364/9240364 [==============================] - 22s - loss: 1.5840 - val_loss: 1.6957
Epoch 3/8
9240364/9240364 [==============================] - 20s - loss: 1.5497 - val_loss: 1.6889
Epoch 4/8
9240364/9240364 [==============================] - 22s - loss: 1.5247 - val_loss: 1.6801
Epoch 5/8
9240364/9240364 [==============================] - 21s - loss: 1.5039 - val_loss: 1.6743
Epoch 6/8
9240364/9240364 [==============================] - 21s - loss: 1.4857 - val_loss: 1.6722
Epoch 7/8
9240364/9240364 [==============================] - 22s - loss: 1.4702 - val_loss: 1.6717
Epoch 8/8
9240364/9240364 [==============================] - 20s - loss: 1.4571 - val_loss: 1.6686


In [41]:
nn23.lr=.0005

In [42]:
nn23.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_anime_scaled, batch_size=20120, epochs=4, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_anime_scaled))

Train on 9240364 samples, validate on 1210873 samples
Epoch 1/4
9240364/9240364 [==============================] - 9s - loss: 1.4172 - val_loss: 1.6681
Epoch 2/4
9240364/9240364 [==============================] - 10s - loss: 1.4057 - val_loss: 1.6696
Epoch 3/4
9240364/9240364 [==============================] - 9s - loss: 1.3994 - val_loss: 1.6703
Epoch 4/4
9240364/9240364 [==============================] - 9s - loss: 1.3953 - val_loss: 1.6719


In [43]:
nn23.save_weights('nn_score_anime_weights.h5')

In [44]:
nn23pred= nn23.predict([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb])
nn23targ = ratings_score_test.score_anime_scaled.values
mean_absolute_error(nn23targ, nn23pred)

0.96388360887655522